# SEC EDGAR Scraping Notebook

@author: Rowan Pan

## Import BeautifulSoup

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd
import numpy as np

In [3]:
base_url = r"https://www.sec.gov/Archives/edgar/data"

## Enter CIK Number HERE

In [4]:
cik_num = '/902012/' # Bill Gates
#'/1043298/' Jeff Bezos

In [5]:
filings_url = base_url + cik_num + "/index.json"

In [6]:
content = requests.get(filings_url)
decoded_content = content.json()
# decoded_content
# no need to print all this out usually

In [7]:
back_up = decoded_content['directory']['item']

## These indexes are the entries from 2017 and after

In [8]:
after_2017 = [i for i in np.arange(len(back_up)) if int(decoded_content['directory']['item'][i]['last-modified'][0:4]) > 2016]

In [9]:
len(after_2017)

18

In [10]:
len(decoded_content['directory']['item'])

984

In [11]:
# cik number again
cik_num = '/902012/'

filings_url = base_url + cik_num + "/index.json"

content = requests.get(filings_url)
decoded_content = content.json()

url_list = []

for filing in decoded_content['directory']['item'][0:len(after_2017)]:
    
    filing_num = filing['name']
    
    filing_url = base_url + cik_num + filing_num + "/index.json"
    
    content = requests.get(filing_url)
    document_content = content.json()
    
    # print all the url's for this CIK
    
    for document in document_content['directory']['item'][0:len(after_2017)]:
 
        if 'xml' in document['name']:
 
            doc_name = document['name']
            document_url = base_url + cik_num + filing_num + '/' + doc_name
            print(document_url)
            url_list.append(document_url)

https://www.sec.gov/Archives/edgar/data/902012/000110465920073034/a4.xml
https://www.sec.gov/Archives/edgar/data/902012/000110465920071037/a3.xml
https://www.sec.gov/Archives/edgar/data/902012/000110465919049112/a3.xml
https://www.sec.gov/Archives/edgar/data/902012/000110465918018418/a4.xml
https://www.sec.gov/Archives/edgar/data/902012/000110465918017529/a4.xml
https://www.sec.gov/Archives/edgar/data/902012/000122520818005874/doc4.xml
https://www.sec.gov/Archives/edgar/data/902012/000110465917070230/a4.xml
https://www.sec.gov/Archives/edgar/data/902012/000122520817016037/doc3.xml
https://www.sec.gov/Archives/edgar/data/902012/000122520817013968/doc5.xml
https://www.sec.gov/Archives/edgar/data/902012/000122520817013341/doc4.xml
https://www.sec.gov/Archives/edgar/data/902012/000122520817009254/doc4.xml
https://www.sec.gov/Archives/edgar/data/902012/000122520817009147/doc4.xml
https://www.sec.gov/Archives/edgar/data/902012/000122520817009034/doc4.xml
https://www.sec.gov/Archives/edgar/da

In [12]:
len(url_list)

16

In [13]:
soup = []

for url in url_list:
    xml_data = requests.get(url).content
    soup.append(BeautifulSoup(xml_data, 'xml'))

In [14]:
len(soup)

16

In [15]:
shares = []
form = []
dates = []
price = []
shares_after = []
ad_code = []
title = []
city = []
name = []

for i in np.arange(len(soup)):
    if soup[i].find('documentType').text == '4'and (
        soup[i].find('securityTitle').text.strip()!= 'Convertible Promissory Note'):
        for result in soup[i].find_all('transactionShares'):
            shares.append(result.text.strip())
        for result in soup[i].find_all('transactionFormType'):
            form.append(result.text.strip())
        for result in soup[i].find_all('transactionDate'):
            dates.append(result.text.strip())
        for result in soup[i].find_all('transactionPricePerShare'):
            price.append(result.text.strip())
        for result in soup[i].find_all('sharesOwnedFollowingTransaction'):
            shares_after.append(result.text.strip())
        for result in soup[i].find_all('transactionAcquiredDisposedCode'):
            ad_code.append(result.text.strip())
        for result in soup[i].find_all('securityTitle'):
            title.append(result.text.strip())
        
        # for later
        for result in soup[i].find_all('rptOwnerCity'):
            city.append(result.text.strip())
        for result in soup[i].find_all('rptOwnerName'):
            name.append(result.text.strip())
        
    
    

In [16]:
len(shares_after), len(form), len(dates), len(ad_code), len(price), len(title), len(shares)

(29, 29, 29, 29, 29, 29, 29)

In [17]:
df = pd.DataFrame({
    "Dates" : dates,
    "Form" : form,
    "Shares" : shares,
    "Shares After": shares_after,
    "Price" : price,
    "A/D Code": ad_code,
    "Security Title": title
})

In [18]:
df.insert(0, 'Name', max(set(name), key = name.count))
df.insert(1, 'City', max(set(city), key = city.count))

In [19]:
# there can be multiple cities, like for Bill Gates
soup[5].find_all('rptOwnerCity')

[<rptOwnerCity>REDMOND</rptOwnerCity>, <rptOwnerCity>KIRKLAND</rptOwnerCity>]

In [20]:
# also can be multiple report owners
soup[5].find_all('rptOwnerName')

[<rptOwnerName>GATES WILLIAM H III</rptOwnerName>,
 <rptOwnerName>CASCADE INVESTMENT, L.L.C.</rptOwnerName>]

In [21]:
# most common city 
max(set(city), key = city.count)

'REDMOND'

In [22]:
# most common name
max(set(name), key = name.count)

'GATES WILLIAM H III'

In [23]:
df

,Name,City,Dates,Form,Shares,Shares After,Price,A/D Code,Security Title
0,GATES WILLIAM H III,REDMOND,2020-06-11,4,6771076,6771076,,A,Common stock
1,GATES WILLIAM H III,REDMOND,2020-06-11,4,2785365,0,,D,Series G preferred stock
2,GATES WILLIAM H III,REDMOND,2020-06-11,4,600173,0,,D,Series H preferred stock
3,GATES WILLIAM H III,REDMOND,2018-03-14,4,15552,30209607,137.2626,A,Common Stock
4,GATES WILLIAM H III,REDMOND,2018-03-14,4,98216,30307823,136.8334,A,Common Stock
5,GATES WILLIAM H III,REDMOND,2018-03-15,4,301931,30609754,136.6253,A,Common Stock
6,GATES WILLIAM H III,REDMOND,2018-03-16,4,32300,30642054,137.9351,A,Common Stock
7,GATES WILLIAM H III,REDMOND,2018-03-16,4,43500,30685554,137.2272,A,Common Stock
8,GATES WILLIAM H III,REDMOND,2018-03-12,4,20540,29845595,136.9232,A,Common Stock
9,GATES WILLIAM H III,REDMOND,2018-03-12,4,79460,29925055,135.8657,A,Common Stock


## Export to Excel

In [24]:
#df.to_excel("Bill_Gates.xlsx",sheet_name='Transactions', index = False)